In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('bank-full.csv', sep=";")
df.head()
df.shape

(45211, 17)

In [4]:
df.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

# ONE HOT ENCODING: https://www.kaggle.com/alexisbcook/categorical-variables

In [5]:
# il one hot encoding necessita della vettorizzazione del dataset
s = (df.dtypes == 'object') #vettorizzo
object_cols = list(s[s].index)

# #SIstemo le categoriche continue
# object_cols = df.select_dtypes(object)


df_OHE = pd.get_dummies(df) #one hot encoding
df_OHE.head()

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,y_no,y_yes
0,58,2143,5,261,1,-1,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0
1,44,29,5,151,1,-1,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0
2,33,2,5,76,1,-1,0,0,0,1,...,1,0,0,0,0,0,0,1,1,0
3,47,1506,5,92,1,-1,0,0,1,0,...,1,0,0,0,0,0,0,1,1,0
4,33,1,5,198,1,-1,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0


# CREO LA X E LA y

In [6]:
y = df_OHE.y_yes

X = df_OHE.drop(['y_yes', "y_no"], axis=1)
print(X.shape)
print(y.shape)

(45211, 51)
(45211,)


# TRAIN_TEST SPLIT: https://www.kaggle.com/thanethunder/exercise-model-validation/edit

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, roc_curve

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
print(train_y.shape)
print(val_y.shape)

(33908,)
(11303,)


# LOGISTIC REGRESSION

In [38]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression(random_state=0)
logistic_model.fit(train_X, train_y)
df_preds = logistic_model.predict(val_X)
print(precision_score(val_y, df_preds))
print(roc_curve(val_y, df_preds))


# >>> from sklearn.datasets import load_iris
# >>> from sklearn.linear_model import LogisticRegression
# >>> X, y = load_iris(return_X_y=True)
# >>> clf = LogisticRegression(random_state=0, solver='lbfgs',
# ...                          multi_class='multinomial').fit(X, y)
# >>> clf.predict(X[:2, :])
# array([0, 0])
# >>> clf.predict_pr*oba(X[:2, :]) 
# array([[9.8...e-01, 1.8...e-02, 1.4...e-08],
#        [9.7...e-01, 2.8...e-02, ...e-08]])
# >>> clf.score(X, y)
# 0.97...

/home/dami/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6318758815232722
(array([0.        , 0.02610522, 1.        ]), array([0.        , 0.34329502, 1.        ]), array([2, 1, 0]))


In [9]:
from sklearn.metrics import confusion_matrix
 
cm = confusion_matrix(val_y, df_preds)
print(cm)

[[9737  261]
 [ 857  448]]


# RANDOM FOREST 1: https://www.kaggle.com/dansbecker/random-forestst

In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score

forest_model = RandomForestClassifier(random_state=0, max_leaf_nodes=5, n_estimators = 10)
forest_model.fit(train_X, train_y)
df_preds = forest_model.predict(val_X)
print(precision_score(val_y, df_preds))

0.828125


In [42]:
from sklearn.metrics import confusion_matrix
 
cm = confusion_matrix(val_y, df_preds)
print(cm)

[[9987   11]
 [1252   53]]


In [15]:
989/(987+142)

0.8759964570416298

# Ma che cazzo èè??????? Se metto  max_leaf_nodes=5 come iperparametro la precisione scatta al 100%?????? =)


 Infatti si è rivelato un modello del cazzo

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_pipeline = Pipeline(steps=[('preprocessor', SimpleImputer()),
                              ('model', RandomForestRegressor(n_estimators=50,
                                                              random_state=0))
                             ])

In [18]:
y = df_OHE.y_yes

X = df_OHE.drop(['y_yes', "y_no"], axis=1)

from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)

MAE scores:
 [0.13641989 0.14190265 0.13373894 0.12836283 0.13745575]


# RANDOM FOREST 2: UNDERFITTING AND OVERFITTING\
https://www.kaggle.com/dansbecker/underfitting-and-overfitting


In [8]:
# def get_precision(max_leaf_nodes, train_X, val_X, train_y, val_y):
#     model = RandomForestClassifier(max_leaf_nodes=max_leaf_nodes, random_state=0)
#     model.fit(train_X, train_y)
#     preds_val = model.predict(val_X)
#     precision = precision_score(val_y, preds_val)
#     return(precision)

# # compare Precisionscore with differing values of max_leaf_nodes
# for max_leaf_nodes in [5, 50, 500, 5000]:
#     my_precision = get_precision(max_leaf_nodes, train_X, val_X, train_y, val_y)
#     print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_precision))

# ADESSO TENTERò DI INTERPRETARE IL MODELLO TRAMITE: 
- Permutation Importance
- Partial Plot

# - Permutation Importance



In [43]:
import eli5


In [44]:
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(forest_model, random_state=1).fit(val_X, val_y)
eli5.show_weights(perm, feature_names = val_X.columns.tolist())

Weight,Feature
0.0036 ± 0.0003,poutcome_success
0.0029 ± 0.0002,poutcome_unknown
0.0026 ± 0.0004,pdays
0.0019 ± 0.0002,previous
0.0018 ± 0.0002,month_sep
0.0017 ± 0.0004,duration
0.0013 ± 0.0005,month_may
0.0012 ± 0.0001,age
0.0012 ± 0.0003,contact_unknown
0.0002 ± 0.0001,contact_cellular
